In [4]:
from euroleague_api import shot_data, game_stats, play_by_play_data, boxscore_data
import pandas as pd
import numpy as np
import re
import json
import copy


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option("max_colwidth", None)
pd.set_option('display.width', 1000)

season_start = 2019
season_end = 2022
game_code = 54

# df = boxscore_data.get_game_boxscore_quarter_data(season, game_code, boxscore_type='EndOfQuarter')
# Assuming BoxScoreData requires initialization, and assuming no parameters are needed for the initialization
boxscore_data_instance = boxscore_data.BoxScoreData()

# Now use the instance to call your method
df = boxscore_data_instance.get_player_boxscore_stats_multiple_seasons(start_season=season_start, end_season=season_end)
# df_season_player = boxscore_data.get_player_boxscore_stats_single_season(season)
# df_game_report = game_stats.get_game_report(season, game_code)
# df_pbp = play_by_play_data.get_game_play_by_play_data(season, game_code)

Season loop:   0%|          | 0/4 [00:00<?, ?it/s]

Season 2019:   0%|          | 0/252 [00:00<?, ?it/s]

Season 2020:   0%|          | 0/328 [00:00<?, ?it/s]

Season 2021:   0%|          | 0/299 [00:00<?, ?it/s]

Season 2022:   0%|          | 0/328 [00:00<?, ?it/s]

In [20]:
df.shape

(33466, 31)

In [36]:
starters_df = df.loc[df['IsStarter'] == 1.0].copy()

In [37]:
starters_df['Gamecode'] = starters_df['Gamecode'].apply(lambda x: f"{x:03}")

In [38]:
starters_df["game_id"] = "E" +starters_df["Season"].astype(str) + "_" + starters_df["Gamecode"].astype(str)

In [41]:
starters_df.head()

,Season,Phase,Round,Gamecode,Home,Player_ID,IsStarter,IsPlaying,Team,Dorsal,Player,Minutes,Points,FieldGoalsMade2,FieldGoalsAttempted2,FieldGoalsMade3,FieldGoalsAttempted3,FreeThrowsMade,FreeThrowsAttempted,OffensiveRebounds,DefensiveRebounds,TotalRebounds,Assistances,Steals,Turnovers,BlocksFavour,BlocksAgainst,FoulsCommited,FoulsReceived,Valuation,Plusminus,game_id
0,2019,RS,1,001,1,PKVZ,1.0,1.0,KHI,1,"SHVED, ALEXEY",29:19,22,4,7,2,10,8,10,0,2,2,6,0,3,1,0,1,5,19,-2,E2019_001
1,2019,RS,1,001,1,P007056,1.0,0.0,KHI,5,"BOOKER, DEVIN",19:21,4,1,4,0,0,2,2,3,3,6,0,2,1,0,0,2,1,7,-12,E2019_001
2,2019,RS,1,001,1,P005718,1.0,0.0,KHI,6,"TIMMA, JANIS",34:53,17,1,3,5,12,0,0,1,3,4,1,4,1,0,0,5,2,13,0,E2019_001
6,2019,RS,1,001,1,P009748,1.0,1.0,KHI,11,"JEREBKO, JONAS",24:29,14,2,7,2,5,4,6,1,6,7,0,2,1,0,2,2,3,11,6,E2019_001
9,2019,RS,1,001,1,P004863,1.0,0.0,KHI,24,"JOVIC, STEFAN",17:00,0,0,0,0,2,0,0,1,0,1,4,0,1,0,0,1,2,3,1,E2019_001


In [25]:
def starter_lineup(df):
    starters_df = df.copy()

    # Create a dictionary with 'Team' as keys and lists of 'Player' as values
    starting_lineups = starters_df.groupby('Team')['Player'].apply(list).to_dict()
    return starting_lineups 

def detect_lineup(df_pbp, df_starter):
    lineup_df = df_pbp.copy()
    lineup_df['A_LINEUP'] = None
    lineup_df['B_LINEUP'] = None

    starting_lineups = starter_lineup(df_starter)
    current_lineups = copy.deepcopy(starting_lineups)
    teams = list(current_lineups.keys())

    # Additional structure to track pending substitutions
    pending_subs = {team: {'in': [], 'out': []} for team in teams}

    def apply_subs(team):
        # Apply substitutions if there is a match in pending 'in' and 'out'
        while pending_subs[team]['in'] and pending_subs[team]['out']:
            player_out = pending_subs[team]['out'].pop(0)
            player_in = pending_subs[team]['in'].pop(0)
            current_lineups[team].remove(player_out)
            current_lineups[team].append(player_in)

    for index, row in lineup_df.iterrows():
        team, action, player = row['CODETEAM'], row['PLAY_ACTION'], row['PLAYER']

        if pd.isna(team) or team not in teams:
            lineup_df.at[index, 'A_LINEUP'] = current_lineups[teams[0]].copy()
            lineup_df.at[index, 'B_LINEUP'] = current_lineups[teams[1]].copy()
            continue

        if action == 'Out':
            # Queue the player to be substituted out
            pending_subs[team]['out'].append(player)
        elif action == 'In':
            # Queue the player to be substituted in
            pending_subs[team]['in'].append(player)

        # Attempt to apply any pending substitutions that can be matched
        apply_subs(team)

        # Update the DataFrame with the current lineup
        lineup_df.at[index, 'A_LINEUP'] = current_lineups[teams[0]].copy()
        lineup_df.at[index, 'B_LINEUP'] = current_lineups[teams[1]].copy()

    return lineup_df

In [26]:
x = detect_lineup(df_pbp, starters_df)

KeyError: 'CODETEAM'

In [3]:
def update_stint_row(df_stints,stint_index, players, current_home, current_away,possessions,home_points,away_points,row):
    
    df_stints.loc[stint_index, players] = 0

    for player in players:
        if player in current_away:
            df_stints.at[stint_index, player] = -1
        elif player in current_home:
            df_stints.at[stint_index, player] = 1
        
        # Update Margin - assuming you want to normalize it per 100 possessions
        df_stints.at[stint_index, 'Margin'] = 100 * (home_points - away_points) / possessions
        df_stints.at[stint_index, 'home_points'] = home_points
        df_stints.at[stint_index, 'away_points'] = away_points
        df_stints.at[stint_index, 'possession'] = possessions
        df_stints.at[stint_index, 'time'] = row["clock"]
        df_stints.at[stint_index, 'HOME'] = row['current_home_lineup']
        df_stints.at[stint_index, 'AWAY'] = row['current_away_lineup']

def process_game_stints(shots, home_starter, away_starter, players):
    # Initialize variables outside of the loop
    possessions = home_score = away_score = home_points = away_points = 0
    stint_index = 0
    df_stints = pd.DataFrame(columns=players + ['Margin', 'home_points', 'away_points', 'possession', 'time', 'HOME', 'AWAY'])
    current_game = shots.iloc[0]["game_id"]
    current_home_lineup = set(home_starter)  
    current_away_lineup = set(away_starter)

    for index, row in shots.iterrows():

        if row['game_id'] != current_game:
            if possessions > 0:
                update_stint_row(df_stints, stint_index, players, current_home_lineup, current_away_lineup, possessions, home_points, away_points, row)
                print(f"{stint_index}-stint updated for game change")
            
            # Reset for new game
            home_score = away_score = home_points = away_points = possessions = 0
            current_game = row["game_id"]
            current_home_lineup = set(row['current_home_lineup'])
            current_away_lineup = set(row['current_away_lineup'])
        # Update current lineups if there has been a change
        if set(row['current_home_lineup']) != current_home_lineup or set(row['current_away_lineup']) != current_away_lineup:
            # Update the stint information before resetting if it's not the start of the game
            
            if 'Serbest Atış 2/3' in row['desc']:
                if row['home_score'] != home_score:
                    home_score += 1
                    home_points += 1
                     
                elif row['away_score'] != away_score:
                    away_score += 1
                    away_points += 1
                    
            elif 'Serbest Atış 2/2' in row['desc'] or 'Serbest Atış 3/3' in row['desc']:
                if row['home_score'] != home_score:
                    home_score += 1
                    home_points += 1
                    possessions += 1
                    update_stint_row(df_stints, stint_index, players, current_home_lineup, current_away_lineup, possessions, home_points, away_points,row)
                    print(f"{stint_index}-stintupdated1")
                    stint_index += 1  # Increment the stint index after updating
                    possessions = 0
                    home_points = 0
                    away_points = 0
                    current_home_lineup = set(row['current_home_lineup'])
                    current_away_lineup = set(row['current_away_lineup'])
                     
                elif row['away_score'] != away_score:
                    away_score += 1
                    away_points += 1
                    possessions += 1
                    update_stint_row(df_stints, stint_index, players, current_home_lineup, current_away_lineup, possessions, home_points, away_points,row)
                    stint_index += 1  # Increment the stint index after updating
                    print(f"{stint_index}-stintupdated2")
                    possessions = 0
                    home_points = 0
                    away_points = 0
                    current_home_lineup = set(row['current_home_lineup'])
                    current_away_lineup = set(row['current_away_lineup'])
                
            else:
                # End the current stint before updating the lineup
                if possessions > 0:
                    update_stint_row(df_stints, stint_index, players, current_home_lineup, current_away_lineup, possessions, home_points, away_points,row)
                    stint_index += 1  # Increment the stint index after updating
                    print(f"{stint_index}-stintupdated3")
                    
                possessions = 0
                home_points = 0
                away_points = 0
                # Update the current game ID
    
                current_home_lineup = set(row['current_home_lineup'])
                current_away_lineup = set(row['current_away_lineup'])
                
    
        # Process the event types and update scores and possessions
        if row['eventType'] == '2pt' or row['eventType'] == '3pt':
            possessions += 1
            point_increment = 3 if row['eventType'] == '3pt' else 2
    
            if row['home_score'] != home_score:
                home_score += point_increment
                home_points += point_increment
    
            if row['away_score'] != away_score:
                away_score += point_increment
                away_points += point_increment
    
        elif row['eventType'] == 'turnover' or row['desc'] == 'Savunma Ribaundu':
            possessions += 1
    
        elif row['eventType'] == 'freeThrow':
            if 'Serbest Atış 1/1' in row['desc'] or 'Serbest Atış 2/2' in row['desc'] or 'Serbest Atış 3/3' in row['desc']:
                possessions += 1
    
                if row['home_score'] != home_score:
                    home_score += 1
                    home_points += 1
                elif row['away_score'] != away_score:
                    away_score += 1
                    away_points += 1
            else:
                # Mid-series free throws, do not increment possessions yet
                if row['home_score'] != home_score:
                    home_score += 1
                    home_points += 1
                elif row['away_score'] != away_score:
                    away_score += 1
                    away_points += 1
    if possessions > 0:
        update_stint_row(df_stints, stint_index, players, current_home_lineup, current_away_lineup, possessions, home_points, away_points,row)
        print(f"{stint_index}-stintupdated4")
    return df_stints

In [7]:
df_pbp = pd.read_csv("../../datasets/basketball/euroleague_play_by_play.csv")
df_pbp.head()

,game_play_id,game_id,game,round,phase,season_code,quarter,type,number_of_play,team_id,player_id,play_type,player,team,dorsal,minute,marker_time,points_a,points_b,comment,play_info
0,E2007_001_002,E2007_001,OLY-BAS,1,REGULAR SEASON,E2007,q1,0,2,BAS,PCBX,3FGM,"PRIGIONI, PABLO",BASKONIA,5.0,1,09:00,0.0,3.0,NaN,Three Pointer (1/1 - 3 pt)
1,E2007_001_003,E2007_001,OLY-BAS,1,REGULAR SEASON,E2007,q1,0,3,BAS,PCBY,AS,"SPLITTER, TIAGO",BASKONIA,21.0,1,09:00,NaN,NaN,NaN,Assist (1)
2,E2007_001_004,E2007_001,OLY-BAS,1,REGULAR SEASON,E2007,q1,0,4,OLY,PTFV,2FGA,"WOODS, QYNTEL",OLYMPIACOS PIRAEUS B.C.,6.0,1,09:00,NaN,NaN,NaN,Missed Two Pointer (0/1 - 0 pt)
3,E2007_001_005,E2007_001,OLY-BAS,1,REGULAR SEASON,E2007,q1,0,5,BAS,PJZJ,D,"TELETOVIC, MIRZA",BASKONIA,12.0,1,09:00,NaN,NaN,NaN,Def Rebound (1)
4,E2007_001_006,E2007_001,OLY-BAS,1,REGULAR SEASON,E2007,q1,0,6,BAS,PCBY,TO,"SPLITTER, TIAGO",BASKONIA,21.0,1,09:00,NaN,NaN,NaN,Turnover (1)


In [11]:
df_pbp.shape

(2031290, 21)

In [6]:
df["play_type"].value_counts()

play_type
OUT         211902
IN          211901
D           185105
RV          163534
CM          150311
AS          127581
2FGM        116710
3FGA        112948
2FGA        111764
FTM         110051
TO          102463
O            80290
3FGM         64406
ST           53040
FTA          35409
LAYUPMD      34085
TOUT         26417
LAYUPATT     26029
FV           20599
AG           20576
EP           12312
OF           11282
BP            9071
TOUT_TV       7855
DUNK          7079
2FGAB         5897
EG            3973
TPOFF         2320
CMU           1933
CMT           1523
JB            1206
3FGAB          687
C              462
CCH            309
B              184
CMD             56
F               14
CMTI             4
BF               2
Name: count, dtype: int64

In [8]:
df["play_info"].unique()

array(['Three Pointer (1/1 -  3 pt)', 'Assist (1)',
       'Missed Two Pointer (0/1 -  0 pt)', ...,
       'Missed Three Pointer (7/13 -  28 pt)',
       'Three Pointer (2/7 -  28 pt)',
       'Missed Three Pointer (2/8 -  29 pt)'], dtype=object)

In [9]:
df.shape

(2031290, 21)

In [15]:
df.season_code.value_counts()

season_code
E2022    171870
E2020    169519
E2021    151766
E2017    134886
E2018    134407
E2016    133517
E2019    132697
E2014    129952
E2015    129422
E2013    126107
E2012    125998
E2007    113072
E2010     96043
E2011     94561
E2009     94433
E2008     93038
Name: count, dtype: int64

In [14]:
df_pbp = df_pbp.loc[df_pbp.season_code.isin(wanted_seasons)]

In [13]:
wanted_seasons = ["E2022","E2020","E2021","E2019"]

In [15]:
df_pbp.shape

(625852, 21)

In [16]:
df_pbp

,game_play_id,game_id,game,round,phase,season_code,quarter,type,number_of_play,team_id,player_id,play_type,player,team,dorsal,minute,marker_time,points_a,points_b,comment,play_info
1405438,E2019_001_002,E2019_001,KHI-TEL,1,REGULAR SEASON,E2019,q1,0,2,NaN,NaN,BP,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,Begin Period
1405439,E2019_001_003,E2019_001,KHI-TEL,1,REGULAR SEASON,E2019,q1,0,3,KHI,P007056,TPOFF,"BOOKER, DEVIN",Khimki Moscow Region,5.0,1,10:00,NaN,NaN,NaN,NaN
1405440,E2019_001_004,E2019_001,KHI-TEL,1,REGULAR SEASON,E2019,q1,0,4,TEL,P008991,TPOFF,"BLACK, TARIK",Maccabi FOX Tel Aviv,28.0,1,10:00,NaN,NaN,NaN,NaN
1405441,E2019_001_005,E2019_001,KHI-TEL,1,REGULAR SEASON,E2019,q1,0,5,KHI,NaN,JB,NaN,Khimki Moscow Region,NaN,1,09:59,NaN,NaN,NaN,NaN
1405442,E2019_001_006,E2019_001,KHI-TEL,1,REGULAR SEASON,E2019,q1,0,6,TEL,P009846,CM,"BRYANT, ELIJAH",Maccabi FOX Tel Aviv,0.0,1,09:56,NaN,NaN,NaN,Foul (1)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2031285,E2022_330_487,E2022_330,OLY-MAD,41,FINAL FOUR,E2022,q4,0,487,MAD,PCVM,OUT,"RODRIGUEZ, SERGIO",Real Madrid,13.0,40,00:03,NaN,NaN,NaN,Out
2031286,E2022_330_488,E2022_330,OLY-MAD,41,FINAL FOUR,E2022,q4,0,488,MAD,PTGB,OUT,"LLULL, SERGIO",Real Madrid,23.0,40,00:03,NaN,NaN,NaN,Out
2031287,E2022_330_489,E2022_330,OLY-MAD,41,FINAL FOUR,E2022,q4,0,489,OLY,P001926,2FGA,"SLOUKAS, KOSTAS",Olympiacos Piraeus,11.0,40,00:02,NaN,NaN,NaN,Missed Two Pointer (1/2 - 6 pt)
2031288,E2022_330_490,E2022_330,OLY-MAD,41,FINAL FOUR,E2022,q4,0,490,MAD,P007982,D,"WILLIAMS-GOSS, NIGEL",Real Madrid,0.0,40,00:01,NaN,NaN,NaN,Def Rebound (2)
